In [6]:
import os

from pathlib import Path

import pandas as pd

from star_parser import StarParser, StarTabDf

In [7]:
data_folder = Path(r'/mnt/DATA/andrea/AM_20211112_NeCen_BRCA1AUb/cryolo_pick')
file_pattern = '*patch_aligned_doseweighted.star'
mic_prefix = ""
mic_prefix_remove = 0
mic_suffix_add = ""
mic_suffix_remove = ""
star_out = "cryolo_pick_all.star"

In [8]:

starfiles = data_folder.glob(f'{file_pattern}')
collated = []
for star in starfiles:
    data = StarParser(star).parse()['data_']
    mic_name = mic_prefix + star.name.replace(mic_suffix_remove,"").replace('.star',f'{mic_suffix_add}.mrc')
    data.fill_column('MicrographName', mic_name, create=True, store=True)
    data.remove_columns(['ClassNumber',	'AnglePsi',	'AutopickFigureOfMerit'], store=True)
    new_order = [data.columns()[-1]] + data.columns()[:-1]
    data.reorder_columns(new_order, store=True)
    collated.append(data.to_df())

In [9]:
merged = pd.concat(collated)
collated_star = StarTabDf(merged)

In [10]:
collated_star.to_df()

,MicrographName,CoordinateX,CoordinateY
0,008479271629128263535_FoilHole_12390118_Data_1...,543.0,3977.0
1,008479271629128263535_FoilHole_12390118_Data_1...,1166.0,3989.0
2,008479271629128263535_FoilHole_12390118_Data_1...,1371.0,4003.0
3,008479271629128263535_FoilHole_12390118_Data_1...,3409.0,3975.0
4,008479271629128263535_FoilHole_12390118_Data_1...,2124.0,3939.0
...,...,...,...
1380792,000989967168210663599_FoilHole_12383667_Data_1...,1480.0,456.0
1380793,000989967168210663599_FoilHole_12383667_Data_1...,473.0,287.0
1380794,000989967168210663599_FoilHole_12383667_Data_1...,1963.0,172.0
1380795,000989967168210663599_FoilHole_12383667_Data_1...,294.0,110.0


In [12]:
with open(data_folder/star_out, 'w') as f:
    f.write(collated_star.to_star())